In [1]:
import pandas as pd

# CSV 파일 로드
file_path = './재정렬된_데이터.csv'
df = pd.read_csv(file_path, encoding='utf-8-sig')

# 주소 중복 확인
duplicated_addresses = df[df.duplicated(subset='주소', keep=False)]

if not duplicated_addresses.empty:
    print("중복된 주소가 발견되었습니다:")
    print(duplicated_addresses)
else:
    print("중복된 주소가 없습니다.")

중복된 주소가 없습니다.


In [3]:
!pip install cx_Oracle


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import cx_Oracle

# 데이터 로드
file_path = './재정렬된_데이터.csv'
df = pd.read_csv(file_path, encoding='utf-8-sig')

# Oracle DB 연결 정보
dsn = cx_Oracle.makedsn("localhost", "1521", service_name="XE")
connection = cx_Oracle.connect(user="C##SCOTT", password="tiger", dsn=dsn)

# Cursor 생성
cursor = connection.cursor()

# 테이블에 데이터를 삽입하는 함수
def insert_data(row):
    try:
        query = """
            INSERT INTO AI_LEARN (
                ADDRESS, PARK_NAME, PARK_DIST,
                HOS_NAME, HOS_DIST,
                SUB_NAME, SUB_DIST,
                BUS_NAME, BUS_DIST,
                PARKING_NAME, PARKING_DIST,
                LOW_NAME, LOW_DIST,
                MID_NAME, MID_DIST,
                HI_NAME, HI_DIST
            ) VALUES (
                :1, :2, :3, 
                :4, :5, 
                :6, :7, 
                :8, :9, 
                :10, :11, 
                :12, :13, 
                :14, :15, 
                :16, :17
            )
        """
        cursor.execute(query, (
            row['주소'], 
            row['공원_요소명'], 
            row['공원_거리 (m)'], 
            row['기관명_요소명'], 
            row['기관명_거리 (m)'], 
            row['역명_요소명'], 
            row['역명_거리 (m)'], 
            row['정류장명_요소명'], 
            row['정류장명_거리 (m)'], 
            row['주차장_요소명'], 
            row['주차장_거리 (m)'], 
            row['초등학교명'], 
            row['초등학교 거리'], 
            row['중학교명'], 
            row['중학교 거리'], 
            row['고등학교명'], 
            row['고등학교 거리']
        ))
    except cx_Oracle.IntegrityError:
        print(f"중복 데이터로 인해 삽입 실패: {row['주소']}")
    except Exception as e:
        print(f"오류 발생: {e}")

# DataFrame의 각 행 데이터를 삽입
for _, row in df.iterrows():
    insert_data(row)

# DB에 커밋
connection.commit()

# 연결 종료
cursor.close()
connection.close()

print("데이터 삽입 완료")


데이터 삽입 완료


In [5]:
import pandas as pd
import cx_Oracle

# 데이터 로드
file_path = './서울시 주차장 최종.csv'
df = pd.read_csv(file_path, encoding='utf-8-sig')
df.head()

,주차장명,위도,경도,주소,전화번호,유무료구분,기본 주차 요금,추가 단위 요금,기본 주차 시간(분 단위),추가 단위 시간(분 단위),일 최대 요금,버스 기본 주차 요금,버스 추가 단위 요금,주소_정제
0,영등포여고 공영(구),37.515205,126.915116,영등포구 신길동 184-3,02-2677-1401,Y,50.0,50.0,5.0,5.0,0.0,0.0,0.0,영등포구 신길동 184-3
1,휘경마을 공영주차장(구),37.588069,127.058093,동대문구 휘경동 308-32,02-2247-9664~6,Y,1000.0,0.0,60.0,0.0,10000.0,0.0,0.0,동대문구 휘경동 308-32
2,압구정 428 공영주차장(구),37.527017,127.026471,강남구 압구정동 428-0,02-2176-0971,Y,400.0,400.0,5.0,5.0,NaN,0.0,0.0,강남구 압구정동 428-0
3,포이초교 공영주차장(구),37.475646,127.052365,강남구 개포동 1273-0,02-2176-0925,Y,200.0,200.0,5.0,5.0,NaN,0.0,0.0,강남구 개포동 1273-0
4,구로디지털단지역 공영주차장(시),37.485432,126.901243,구로구 구로동 810-3,02-2290-6034,Y,260.0,260.0,5.0,5.0,18700.0,0.0,0.0,구로구 구로동 810-3


In [6]:
import pandas as pd
import cx_Oracle

# 데이터 로드
file_path = './서울시 주차장 최종.csv'
df = pd.read_csv(file_path, encoding='utf-8-sig')

# NaN 값을 처리 (숫자 컬럼은 0, 문자열 컬럼은 빈 문자열로 대체)
df.fillna({
    '주차장명': '',  # PARKING_NAME
    '위도': 0,  # PARKING_LAT
    '경도': 0,  # PARKING_LON
    '주소': '',  # PARKING_ADDRESS
    '전화번호': '',  # PARKING_CALL
    '유무료구분': '',  # PARKING_FREE
    '기본 주차 요금': 0,  # PARKING_PLANE
    '추가 단위 요금': 0,  # PARKING_MORE
    '기본 주차 시간(분 단위)': 0,  # PARKING_PLANE_TIME
    '추가 단위 시간(분 단위)': 0,  # PARKING_MORE_TIME
    '일 최대 요금': 0,  # PARKING_DMAX
    '버스 기본 주차 요금': 0,  # PARKING_BUS_PLANE
    '버스 추가 단위 요금': 0  # PARKING_BUS_MORE
}, inplace=True)

# Oracle DB 연결 정보
dsn = cx_Oracle.makedsn("localhost", "1521", service_name="XE")
connection = cx_Oracle.connect(user="C##SCOTT", password="tiger", dsn=dsn)

# Cursor 생성
cursor = connection.cursor()

# 테이블에 데이터를 삽입하는 함수
def insert_data(row):
    try:
        query = """
            INSERT INTO PARKING (
                PARKING_NAME, PARKING_LAT, PARKING_LON, PARKING_ADDRESS,
                PARKING_CALL, PARKING_FREE, PARKING_PLANE, PARKING_MORE,
                PARKING_PLANE_TIME, PARKING_MORE_TIME, PARKING_DMAX,
                PARKING_BUS_PLANE, PARKING_BUS_MORE
            ) VALUES (
                :1, :2, :3, :4,
                :5, :6, :7, :8,
                :9, :10, :11, :12, :13
            )
        """
        cursor.execute(query, (
            row['주차장명'], 
            row['위도'], 
            row['경도'], 
            row['주소'], 
            row['전화번호'], 
            row['유무료구분'], 
            row['기본 주차 요금'], 
            row['추가 단위 요금'], 
            row['기본 주차 시간(분 단위)'], 
            row['추가 단위 시간(분 단위)'], 
            row['일 최대 요금'], 
            row['버스 기본 주차 요금'], 
            row['버스 추가 단위 요금']
        ))

    except cx_Oracle.IntegrityError as e:
        print(f"중복 데이터 삽입 오류: {row['주차장명']}, 오류 메시지: {e}")
    except Exception as e:
        print(f"오류 발생: {e}, 데이터: {row}")

# DataFrame의 각 행 데이터를 삽입
for _, row in df.iterrows():
    insert_data(row)

# DB에 커밋
connection.commit()

# 연결 종료
cursor.close()
connection.close()

print("데이터 삽입 완료")


데이터 삽입 완료


In [7]:
import pandas as pd
import cx_Oracle

# CSV 파일 로드
file_path = './서울 공원 결측치_제거.csv'
df = pd.read_csv(file_path, encoding='cp949')

# NaN 값을 처리 (숫자형 컬럼은 0, 문자열 컬럼은 빈 문자열로 대체)
df.fillna({
    '공원명': '',    # PARK_NAME
    '위도': 0,       # PARK_LAT
    '경도': 0,       # PARK_LON
    '이미지': '',    # PARK_IMG
    '지역': '',      # PARK_AREA
    '공원주소': ''   # PARK_ADDRESS
}, inplace=True)

# Oracle DB 연결 정보
dsn = cx_Oracle.makedsn("localhost", "1521", service_name="XE")
connection = cx_Oracle.connect(user="C##SCOTT", password="tiger", dsn=dsn)

# Cursor 생성
cursor = connection.cursor()

# 데이터 삽입 함수
def insert_data(row):
    try:
        query = """
            INSERT INTO PARK (
                PARK_NAME, PARK_LAT, PARK_LON, PARK_IMG, PARK_AREA, PARK_ADDRESS
            ) VALUES (
                :1, :2, :3, :4, :5, :6
            )
        """
        cursor.execute(query, (
            row['공원명'],         # PARK_NAME
            row['위도'],           # PARK_LAT
            row['경도'],           # PARK_LON
            row['이미지'],         # PARK_IMG
            row['지역'],           # PARK_AREA
            row['공원주소']        # PARK_ADDRESS
        ))
    except cx_Oracle.IntegrityError:
        print(f"중복 데이터로 삽입 실패: {row['공원명']}")
    except Exception as e:
        print(f"오류 발생: {e}, 공원명: {row['공원명']}")

# 데이터 삽입 실행
for _, row in df.iterrows():
    insert_data(row)

# 커밋 및 연결 종료
connection.commit()
cursor.close()
connection.close()

print("데이터 삽입 완료!")


데이터 삽입 완료!


In [8]:
import pandas as pd
import cx_Oracle

# CSV 파일 로드
file_path = './지하철_전처리_최종.csv'
df = pd.read_csv(file_path, encoding='utf-8-sig')

# Oracle DB 연결 정보
dsn = cx_Oracle.makedsn("localhost", "1521", service_name="XE")
connection = cx_Oracle.connect(user="C##SCOTT", password="tiger", dsn=dsn)

# Cursor 생성
cursor = connection.cursor()

# 데이터 삽입 함수
def insert_data(row):
    try:
        query = """
            INSERT INTO SUBWAY (
                SUB_NAME, SUB_LAT, SUB_LON
            ) VALUES (
                :1, :2, :3
            )
        """
        cursor.execute(query, (
            row['역명'],  # SUB_NAME
            row['위도'],  # SUB_LAT
            row['경도']   # SUB_LON
        ))
    except cx_Oracle.IntegrityError:
        print(f"중복 데이터로 삽입 실패: {row['역명']}")
    except Exception as e:
        print(f"오류 발생: {e}, 역명: {row['역명']}")

# 데이터 삽입 실행
for _, row in df.iterrows():
    insert_data(row)

# 커밋 및 연결 종료
connection.commit()
cursor.close()
connection.close()

print("데이터 삽입 완료!")


데이터 삽입 완료!


In [11]:
import pandas as pd
import cx_Oracle

# CSV 파일 로드
file_path = './hospital.csv'
df = pd.read_csv(file_path, encoding='utf-8')

# Oracle DB 연결 정보
dsn = cx_Oracle.makedsn("localhost", "1521", service_name="XE")
connection = cx_Oracle.connect(user="C##SCOTT", password="tiger", dsn=dsn)

# Cursor 생성
cursor = connection.cursor()

# 데이터 삽입 함수
def insert_data(row):
    try:
        query = """
            INSERT INTO HOSPITAL (
                HOS_NAME, HOS_LAT, HOS_LON, HOS_ADDRESS,
                HOS_CALL, HOS_S, HOS_C
            ) VALUES (
                :1, :2, :3, :4, :5, :6, :7
            )
        """
        cursor.execute(query, (
            row['기관명'],           # HOS_NAME
            row['병원위도'],         # HOS_LAT
            row['병원경도'],         # HOS_LON
            row['주소'],             # HOS_ADDRESS
            row['대표전화'],         # HOS_CALL
            row['진료시간(월요일)S'], # HOS_S
            row['진료시간(월요일)C']  # HOS_C
        ))
    except cx_Oracle.IntegrityError:
        print(f"중복 데이터로 삽입 실패: {row['기관명']}")
    except Exception as e:
        print(f"오류 발생: {e}, 기관명: {row['기관명']}")

# 데이터 삽입 실행
for _, row in df.iterrows():
    insert_data(row)

# 커밋 및 연결 종료
connection.commit()
cursor.close()
connection.close()

print("데이터 삽입 완료!")


데이터 삽입 완료!


In [12]:
import pandas as pd
import cx_Oracle

# CSV 파일 로드
file_path = './bus_stop.csv'
df = pd.read_csv(file_path, encoding='utf-8')

# Oracle DB 연결 정보
dsn = cx_Oracle.makedsn("localhost", "1521", service_name="XE")
connection = cx_Oracle.connect(user="C##SCOTT", password="tiger", dsn=dsn)

# Cursor 생성
cursor = connection.cursor()

# 데이터 삽입 함수
def insert_data(row):
    try:
        query = """
            INSERT INTO BUS (
                BUS_NAME, BUS_LAT, BUS_LON
            ) VALUES (
                :1, :2, :3
            )
        """
        cursor.execute(query, (
            row['정류장명'],  # BUS_NAME
            row['위도'],     # BUS_LAT
            row['경도']      # BUS_LON
        ))
    except cx_Oracle.IntegrityError:
        print(f"중복 데이터로 삽입 실패: {row['정류장명']}")
    except Exception as e:
        print(f"오류 발생: {e}, 정류장명: {row['정류장명']}")

# 데이터 삽입 실행
for _, row in df.iterrows():
    insert_data(row)

# 커밋 및 연결 종료
connection.commit()
cursor.close()
connection.close()

print("데이터 삽입 완료!")


데이터 삽입 완료!


In [13]:
import pandas as pd
import cx_Oracle

# CSV 파일 로드
file_path = './초등학교.csv'
df = pd.read_csv(file_path, encoding='utf-8-sig')

# Oracle DB 연결 정보
dsn = cx_Oracle.makedsn("localhost", "1521", service_name="XE")
connection = cx_Oracle.connect(user="C##SCOTT", password="tiger", dsn=dsn)

# Cursor 생성
cursor = connection.cursor()

# 데이터 삽입 함수
def insert_data(row):
    try:
        query = """
            INSERT INTO LOWSCHOOLE (
                LOW_NAME, LOW_LAT, LOW_LON, LOW_BUIL, LOW_ADDRESS
            ) VALUES (
                :1, :2, :3, :4, :5
            )
        """
        cursor.execute(query, (
            row['학교명'],      # LOW_NAME
            row['위도'],        # LOW_LAT
            row['경도'],        # LOW_LON
            row['설립형태'],    # LOW_BUIL
            row['소재지도로명주소']  # LOW_ADDRESS
        ))
    except cx_Oracle.IntegrityError:
        print(f"중복 데이터로 삽입 실패: {row['학교명']}")
    except Exception as e:
        print(f"오류 발생: {e}, 학교명: {row['학교명']}")

# 데이터 삽입 실행
for _, row in df.iterrows():
    insert_data(row)

# 커밋 및 연결 종료
connection.commit()
cursor.close()
connection.close()

print("데이터 삽입 완료!")


데이터 삽입 완료!


In [14]:
import pandas as pd
import cx_Oracle

# CSV 파일 로드
file_path = './중학교.csv'
df = pd.read_csv(file_path, encoding='utf-8-sig')

# Oracle DB 연결 정보
dsn = cx_Oracle.makedsn("localhost", "1521", service_name="XE")
connection = cx_Oracle.connect(user="C##SCOTT", password="tiger", dsn=dsn)

# Cursor 생성
cursor = connection.cursor()

# 데이터 삽입 함수
def insert_data(row):
    try:
        query = """
            INSERT INTO MIDDLESCHOOLE (
                MID_NAME, MID_LAT, MID_LON, MID_BUIL, MID_ADDRESS
            ) VALUES (
                :1, :2, :3, :4, :5
            )
        """
        cursor.execute(query, (
            row['학교명'],        # MID_NAME
            row['위도'],          # MID_LAT
            row['경도'],          # MID_LON
            row['설립형태'],      # MID_BUIL
            row['소재지도로명주소'] # MID_ADDRESS
        ))
    except cx_Oracle.IntegrityError:
        print(f"중복 데이터로 삽입 실패: {row['학교명']}")
    except Exception as e:
        print(f"오류 발생: {e}, 학교명: {row['학교명']}")

# 데이터 삽입 실행
for _, row in df.iterrows():
    insert_data(row)

# 커밋 및 연결 종료
connection.commit()
cursor.close()
connection.close()

print("데이터 삽입 완료!")


데이터 삽입 완료!


In [15]:
import pandas as pd
import cx_Oracle

# CSV 파일 로드
file_path = './고등학교.csv'
df = pd.read_csv(file_path, encoding='utf-8-sig')

# Oracle DB 연결 정보
dsn = cx_Oracle.makedsn("localhost", "1521", service_name="XE")
connection = cx_Oracle.connect(user="C##SCOTT", password="tiger", dsn=dsn)

# Cursor 생성
cursor = connection.cursor()

# 데이터 삽입 함수
def insert_data(row):
    try:
        query = """
            INSERT INTO HIGHSCHOOLE (
                HI_NAME, HI_LAT, HI_LON, HI_BUIL, HI_ADDRESS
            ) VALUES (
                :1, :2, :3, :4, :5
            )
        """
        cursor.execute(query, (
            row['학교명'],        # HI_NAME
            row['위도'],          # HI_LAT
            row['경도'],          # HI_LON
            row['설립형태'],      # HI_BUIL
            row['소재지도로명주소'] # HI_ADDRESS
        ))
    except cx_Oracle.IntegrityError:
        print(f"중복 데이터로 삽입 실패: {row['학교명']}")
    except Exception as e:
        print(f"오류 발생: {e}, 학교명: {row['학교명']}")

# 데이터 삽입 실행
for _, row in df.iterrows():
    insert_data(row)

# 커밋 및 연결 종료
connection.commit()
cursor.close()
connection.close()

print("데이터 삽입 완료!")


데이터 삽입 완료!
